In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import cv2
import os
import time
import random

print(tf.__version__)
from tensorflow.keras import optimizers, metrics, losses
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPool2D, UpSampling2D, Concatenate, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras import backend as K
from keras.initializers import RandomNormal

from keras.layers import Input, Conv2D, Flatten, Dense, Conv2DTranspose, Reshape, Lambda, Activation, BatchNormalization, LeakyReLU, Dropout, ZeroPadding2D, UpSampling2D
from tensorflow.keras.layers import Conv1D,MaxPooling1D,Conv1DTranspose,AveragePooling1D,GlobalMaxPool1D,GlobalAveragePooling1D
from tensorflow.keras.layers import Conv2D,MaxPool2D,Conv2DTranspose,AveragePooling2D,GlobalMaxPool2D,GlobalAveragePooling2D
from tensorflow.keras.layers import Conv3D,MaxPooling3D,Conv3DTranspose,AveragePooling3D,GlobalMaxPool3D,GlobalAveragePooling3D
from tensorflow.keras.layers import Dense,Flatten,Dropout,Concatenate,Layer,BatchNormalization,Input,Add,Activation,Average

from tensorflow.keras import layers

2024-11-20 20:42:37.526278: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9373] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-20 20:42:37.526417: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-20 20:42:37.658593: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1534] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2.15.0


In [2]:
#data_path = "outlineOutput"
#data_path = "H:\\download\\blender\\projects\\vdmTests\\outlineOutput"
#output_path = "H:\\tmp\\dexined"
#data_path = "/media/Tomasz/4T/work/dexined/outlineOutput"

data_path = "/media/Tomasz/4T/work/dexined/tmp/largeEdgeDetectionDataset"
output_path = "/media/Tomasz/4T/work/tmp/dexined/allindataset"


train_path = data_path + "/train/*"
edge_train_path = data_path + "/train_edge/*"

test_path = data_path + "/test/*"
edge_test_path = data_path + "/test_edge/*"

val_path = data_path + "/val/*"
edge_val_path = data_path + "/val_edge/*"

print(edge_test_path)

/media/Tomasz/4T/work/dexined/tmp/largeEdgeDetectionDataset/test_edge/*


In [3]:
def load_data(ipath, epath,pattern):
    pattern_ipath = ipath + "*" +pattern+"*"
    pattern_epath = epath + "*" +pattern+"*"
    images = sorted(glob(os.path.join(pattern_ipath)))
    edges = sorted(glob(os.path.join(pattern_epath)))
    images = np.array(images)
    edges = np.array(edges)
    
    indices = np.arange(images.shape[0])
    np.random.shuffle(indices)

    images = images[indices]
    edges = edges[indices]
    return images.tolist(), edges.tolist()

In [4]:
images, edges = load_data(train_path, edge_train_path,"*")
valimg, valedg = load_data(val_path, edge_val_path,"*")

print(len(images), len(edges))
print(len(valimg), len(valedg))

40595 40595
4162 4162


In [5]:
def read_image(path, H=512, W=912):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W,H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_edge(path, H=512, W=912):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (W,H))
    x = np.expand_dims(x, axis=-1)
    x = x/255.0
    x = x.astype(np.float32)
    return x

def preprocess(x,y,H=512, W=912):

    def f(x,y):
        #print(x,"|||",y)
        x = x.decode()
        y = y.decode()

        x = read_image(x)
        y = read_edge(y)
        return x, y

    images, edges = tf.numpy_function(f, [x, y], [tf.float32, tf.float32])
    images.set_shape([H, W, 3])
    edges.set_shape([H, W, 1])
    return images, edges

AUTOTUNE = tf.data.AUTOTUNE

def tf_data(x,y):
    data = tf.data.Dataset.from_tensor_slices((x,y))
    data = data.cache()
    #data = data.shuffle(buffer_size=1000)
    data = data.map(preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    data = data.batch(12)
    data = data.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return data


In [6]:
train_data = tf_data(images, edges)
val_data = tf_data(valimg, valedg)
print(len(train_data),len(val_data))

2024-11-20 20:43:12.124377: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-20 20:43:12.414445: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-20 20:43:12.414730: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-20 20:43:12.416739: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-20 20:43:12.416874: I external/local_xla/xla/stream_executor

3383 347


In [7]:
class GPUMemoryCallback(tf.keras.callbacks.Callback):
    def __init__(self, target_batches, print_stats=False, **kwargs):
        """
        target_batches: A list of batch indices at which to record memory usage.
        print_stats: A boolean flag indicating whether to print memory usage statistics.
        """
        super().__init__(**kwargs)
        self.target_batches = target_batches
        self.print_stats = print_stats

        self.memory_usage = []
        self.labels = []

    def _compute_memory_usage(self):
        memory_stats = tf.config.experimental.get_memory_info("GPU:0")
        # Convert bytes to GB and store in list.
        peak_usage = round(memory_stats["peak"] / (2**30), 3)
        self.memory_usage.append(peak_usage)

    def on_epoch_begin(self, epoch, logs=None):
        self._compute_memory_usage()
        self.labels.append(f"epoch {epoch} start")

    def on_train_batch_begin(self, batch, logs=None):
        if batch in self.target_batches:
            self._compute_memory_usage()
            self.labels.append(f"batch {batch}")

    def on_epoch_end(self, epoch, logs=None):
        self._compute_memory_usage()
        self.labels.append(f"epoch {epoch} end")
        
gpu_memory_callback = GPUMemoryCallback(
    target_batches=[1, 2, 3],
    print_stats=True,
)

In [8]:
model_filepath = output_path + "/dexined-allindataset-{epoch:02d}-{val_binary_accuracy:.4f}.hdf5"
checkpoint =ModelCheckpoint(
    filepath=model_filepath,
    #save_weights_only=True,
    monitor='val_binary_accuracy',
    mode='max',
    save_best_only=False,
    verbose=1)
    

In [9]:
weight_init = tf.initializers.glorot_uniform()

l2 = regularizers.l2
w_decay=1e-3

glorot_normal = RandomNormal(stddev=0.01)

orthogonal = tf.keras.initializers.Orthogonal(
    gain=1.0, seed=None
)



In [ ]:
self_conv2D_16 = Conv2D(32 , kernel_size=(3,3),strides=(2,2),padding = 'same',use_bias=True,kernel_initializer= glorot_normal)
self_batchnormalization_9 = BatchNormalization()
self_conv2D_29 = Conv2D(64 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True,kernel_initializer= glorot_normal)
self_batchnormalization_2 = BatchNormalization()
self_activation_18 = Activation(activation='relu')
self_conv2D_1 = Conv2D(128 , kernel_size=(3,3),strides=(1,1),padding = 'same')
self_conv2D_42 = Conv2D(1 , kernel_size=(1,1),strides=(1,1),padding = 'same',activation='relu',use_bias=True)
self_conv2D_50 = Conv2D(128 , kernel_size=(1,1),strides=(2,2),padding = 'same',use_bias=True)
self_batchnormalization_1 = BatchNormalization()
self_transpoze2D_7 = Conv2DTranspose(1 , kernel_size=(2,2),strides=(2,2),padding = 'same',use_bias=True)
self_conv2D_39 = Conv2D(128 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_18 = BatchNormalization()
self_activation_1 = Activation(activation='relu')
self_conv2D_51 = Conv2D(1 , kernel_size=(1,1),strides=(1,1),padding = 'same',activation='relu',use_bias=True)
self_maxpool2D_1 = MaxPool2D(pool_size=(3,3),strides=(2,2),padding = 'same')
self_transpoze2D_8 = Conv2DTranspose(1 , kernel_size=(2,2),strides=(2,2),padding = 'same',use_bias=True)

self_conv2D_14 = Conv2D(256 , kernel_size=(1,1),strides=(1,1),padding = 'same',use_bias=True)
self_conv2D_24 = Conv2D(256 , kernel_size=(1,1),strides=(2,2),padding = 'same',use_bias=True)
self_activation_16 = Activation(activation='relu')
self_conv2D_37 = Conv2D(256 , kernel_size=(1,1),strides=(2,2),padding = 'same',use_bias=True)
self_conv2D_38 = Conv2D(512 , kernel_size=(1,1),strides=(2,2),padding = 'same')
self_conv2D_20 = Conv2D(256 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_16 = BatchNormalization()
self_activation_17 = Activation(activation='relu')
self_conv2D_18 = Conv2D(256 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_7 = BatchNormalization()

self_activation_14 = Activation(activation='relu')
self_conv2D_17 = Conv2D(256 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_14 = BatchNormalization()
self_activation_15 = Activation(activation='relu')
self_conv2D_2 = Conv2D(256 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_5 = BatchNormalization()

self_conv2D_26 = Conv2D(16 , kernel_size=(1,1),strides=(1,1),padding = 'same',activation='relu',use_bias=True)
self_maxpool2D_2 = MaxPool2D(pool_size=(3,3),strides=(2,2),padding = 'same')
self_transpoze2D_13 = Conv2DTranspose(16 , kernel_size=(4,4),strides=(2,2),padding = 'same',use_bias=True)


self_conv2D_23 = Conv2D(1 , kernel_size=(1,1),strides=(1,1),padding = 'same',activation='relu',use_bias=True)
self_conv2D_21 = Conv2D(512 , kernel_size=(1,1),strides=(2,2),padding = 'same',use_bias=True)
self_conv2D_41 = Conv2D(512 , kernel_size=(1,1),strides=(1,1),padding = 'same',use_bias=True)
self_transpoze2D_14 = Conv2DTranspose(1 , kernel_size=(4,4),strides=(2,2),padding = 'same',use_bias=True)
self_activation_12 = Activation(activation='relu')
self_conv2D_19 = Conv2D(512 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_12 = BatchNormalization()
self_activation_13 = Activation(activation='relu')
self_conv2D_13 = Conv2D(512 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_3 = BatchNormalization()

self_activation_9 = Activation(activation='relu')
self_conv2D_11 = Conv2D(512 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_22 = BatchNormalization()
self_activation_23 = Activation(activation='relu')
self_conv2D_45 = Conv2D(512 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_26 = BatchNormalization()

self_activation_7 = Activation(activation='relu')
self_conv2D_15 = Conv2D(512 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_24 = BatchNormalization()
self_activation_8 = Activation(activation='relu')
self_conv2D_9 = Conv2D(512 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_21 = BatchNormalization()

self_conv2D_33 = Conv2D(16 , kernel_size=(1,1),strides=(1,1),padding = 'same',activation='relu',use_bias=True)
self_maxpool2D_3 = MaxPool2D(pool_size=(3,3),strides=(2,2),padding = 'same')
self_transpoze2D_5 = Conv2DTranspose(16 , kernel_size=(8,8),strides=(2,2),padding = 'same',use_bias=True)


self_conv2D_35 = Conv2D(16 , kernel_size=(1,1),strides=(1,1),padding = 'same',activation='relu',use_bias=True)
self_activation_22 = Activation(activation='relu')
self_conv2D_43 = Conv2D(512 , kernel_size=(1,1),strides=(1,1),padding = 'same',use_bias=True)
self_conv2D_4 = Conv2D(512 , kernel_size=(1,1),strides=(1,1),padding = 'same',use_bias=True)
self_transpoze2D_6 = Conv2DTranspose(16 , kernel_size=(8,8),strides=(2,2),padding = 'same',use_bias=True)
self_conv2D_32 = Conv2D(512 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_conv2D_30 = Conv2D(1 , kernel_size=(1,1),strides=(1,1),padding = 'same',activation='relu',use_bias=True)
self_batchnormalization_17 = BatchNormalization()
self_transpoze2D_12 = Conv2DTranspose(1 , kernel_size=(8,8),strides=(2,2),padding = 'same',use_bias=True)
self_activation_24 = Activation(activation='relu')
self_conv2D_3 = Conv2D(512 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_25 = BatchNormalization()

self_activation_20 = Activation(activation='relu')
self_conv2D_6 = Conv2D(512 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_23 = BatchNormalization()
self_activation_21 = Activation(activation='relu')
self_conv2D_49 = Conv2D(512 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_15 = BatchNormalization()

self_activation_2 = Activation(activation='relu')
self_conv2D_48 = Conv2D(512 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_11 = BatchNormalization()
self_activation_3 = Activation(activation='relu')
self_conv2D_40 = Conv2D(512 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_13 = BatchNormalization()


self_conv2D_22 = Conv2D(16 , kernel_size=(1,1),strides=(1,1),padding = 'same',activation='relu',use_bias=True)
self_conv2D_46 = Conv2D(256 , kernel_size=(1,1),strides=(1,1),padding = 'same',use_bias=True)
self_activation_5 = Activation(activation='relu')
self_transpoze2D_2 = Conv2DTranspose(16 , kernel_size=(16,16),strides=(2,2),padding = 'same',use_bias=True)
self_conv2D_7 = Conv2D(256 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_conv2D_34 = Conv2D(16 , kernel_size=(1,1),strides=(1,1),padding = 'same',activation='relu',use_bias=True)
self_batchnormalization_20 = BatchNormalization()
self_transpoze2D_10 = Conv2DTranspose(16 , kernel_size=(16,16),strides=(2,2),padding = 'same',use_bias=True)
self_activation_6 = Activation(activation='relu')
self_conv2D_25 = Conv2D(16 , kernel_size=(1,1),strides=(1,1),padding = 'same',activation='relu',use_bias=True)
self_conv2D_12 = Conv2D(256 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_transpoze2D_3 = Conv2DTranspose(16 , kernel_size=(16,16),strides=(2,2),padding = 'same',use_bias=True)
self_batchnormalization_6 = BatchNormalization()
self_conv2D_31 = Conv2D(1 , kernel_size=(1,1),strides=(1,1),padding = 'same',activation='relu',use_bias=True)

self_transpoze2D_11 = Conv2DTranspose(1 , kernel_size=(16,16),strides=(2,2),padding = 'same',use_bias=True)
self_activation_10 = Activation(activation='relu')
self_conv2D_5 = Conv2D(256 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_19 = BatchNormalization()
self_activation_11 = Activation(activation='relu')
self_conv2D_8 = Conv2D(256 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_4 = BatchNormalization()

self_activation_4 = Activation(activation='relu')
self_conv2D_44 = Conv2D(256 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_8 = BatchNormalization()
self_activation_19 = Activation(activation='relu')
self_conv2D_10 = Conv2D(256 , kernel_size=(3,3),strides=(1,1),padding = 'same',use_bias=True)
self_batchnormalization_10 = BatchNormalization()

self_conv2D_27 = Conv2D(16 , kernel_size=(1,1),strides=(1,1),padding = 'same',activation='relu',use_bias=True)
self_transpoze2D_9 = Conv2DTranspose(16 , kernel_size=(16,16),strides=(2,2),padding = 'same',use_bias=True)
self_conv2D_28 = Conv2D(16 , kernel_size=(1,1),strides=(1,1),padding = 'same',activation='relu',use_bias=True)
self_transpoze2D_15 = Conv2DTranspose(16 , kernel_size=(16,16),strides=(2,2),padding = 'same',use_bias=True)
self_conv2D_47 = Conv2D(16 , kernel_size=(1,1),strides=(1,1),padding = 'same',activation='relu',use_bias=True)
self_transpoze2D_4 = Conv2DTranspose(16 , kernel_size=(16,16),strides=(2,2),padding = 'same',use_bias=True)
self_conv2D_36 = Conv2D(1 , kernel_size=(1,1),strides=(1,1),padding = 'same',activation='relu',use_bias=True)
self_transpoze2D_1 = Conv2DTranspose(1 , kernel_size=(16,16),strides=(2,2),padding = 'same',use_bias=True)
self_concatenate_1 = Concatenate(axis=3)
self_last_conv = Conv2D(1, kernel_size=(1,1), strides=(1,1), padding = 'same', kernel_initializer=tf.constant_initializer(1/5))




#-------------------------------------

input2D_1 = Input(shape=(512,912,3))

conv2D_16 = self_conv2D_16(input2D_1)
batchnormalization_9 = self_batchnormalization_9(conv2D_16)
conv2D_29 = self_conv2D_29(batchnormalization_9)
batchnormalization_2 = self_batchnormalization_2(conv2D_29)
activation_18 = self_activation_18(batchnormalization_2)
conv2D_1 = self_conv2D_1(activation_18)
conv2D_42 = self_conv2D_42(activation_18)
conv2D_50 = self_conv2D_50(activation_18)
batchnormalization_1 = self_batchnormalization_1(conv2D_1)
transpoze2D_7 = self_transpoze2D_7(conv2D_42)
conv2D_39 = self_conv2D_39(batchnormalization_1)
batchnormalization_18 = self_batchnormalization_18(conv2D_39)
activation_1 = self_activation_1(batchnormalization_18)
conv2D_51 = self_conv2D_51(activation_1)
maxpool2D_1 = self_maxpool2D_1(activation_1)
transpoze2D_8 = self_transpoze2D_8(conv2D_51)
add_1 = Add()([conv2D_50,maxpool2D_1])
conv2D_14 = self_conv2D_14(maxpool2D_1)
conv2D_24 = self_conv2D_24(maxpool2D_1)
activation_16 = self_activation_16(add_1)
conv2D_37 = self_conv2D_37(add_1)
conv2D_38 = self_conv2D_38(conv2D_24)
conv2D_20 = self_conv2D_20(activation_16)
batchnormalization_16 = self_batchnormalization_16(conv2D_20)
activation_17 = self_activation_17(batchnormalization_16)
conv2D_18 = self_conv2D_18(activation_17)
batchnormalization_7 = self_batchnormalization_7(conv2D_18)
average_9 = Average()([batchnormalization_7,conv2D_14])
activation_14 = self_activation_14(average_9)
conv2D_17 = self_conv2D_17(activation_14)
batchnormalization_14 = self_batchnormalization_14(conv2D_17)
activation_15 = self_activation_15(batchnormalization_14)
conv2D_2 = self_conv2D_2(activation_15)
batchnormalization_5 = self_batchnormalization_5(conv2D_2)
average_8 = Average()([batchnormalization_5,conv2D_14])
conv2D_26 = self_conv2D_26(average_8)
maxpool2D_2 = self_maxpool2D_2(average_8)
transpoze2D_13 = self_transpoze2D_13(conv2D_26)
add_2 = Add()([conv2D_37,maxpool2D_2])
add_4 = Add()([conv2D_24,maxpool2D_2])
conv2D_23 = self_conv2D_23(transpoze2D_13)
conv2D_21 = self_conv2D_21(add_2)
conv2D_41 = self_conv2D_41(add_4)
transpoze2D_14 = self_transpoze2D_14(conv2D_23)
activation_12 = self_activation_12(add_2)
conv2D_19 = self_conv2D_19(activation_12)
batchnormalization_12 = self_batchnormalization_12(conv2D_19)
activation_13 = self_activation_13(batchnormalization_12)
conv2D_13 = self_conv2D_13(activation_13)
batchnormalization_3 = self_batchnormalization_3(conv2D_13)
average_7 = Average()([batchnormalization_3,conv2D_41])
activation_9 = self_activation_9(average_7)
conv2D_11 = self_conv2D_11(activation_9)
batchnormalization_22 = self_batchnormalization_22(conv2D_11)
activation_23 = self_activation_23(batchnormalization_22)
conv2D_45 = self_conv2D_45(activation_23)
batchnormalization_26 = self_batchnormalization_26(conv2D_45)
average_6 = Average()([batchnormalization_26,conv2D_41])
activation_7 = self_activation_7(average_6)
conv2D_15 = self_conv2D_15(activation_7)
batchnormalization_24 = self_batchnormalization_24(conv2D_15)
activation_8 = self_activation_8(batchnormalization_24)
conv2D_9 = self_conv2D_9(activation_8)
batchnormalization_21 = self_batchnormalization_21(conv2D_9)
average_5 = Average()([batchnormalization_21,conv2D_41])
conv2D_33 = self_conv2D_33(average_5)
maxpool2D_3 = self_maxpool2D_3(average_5)
transpoze2D_5 = self_transpoze2D_5(conv2D_33)
add_3 = Add()([conv2D_21,maxpool2D_3])
add_5 = Add()([conv2D_38,maxpool2D_3])
conv2D_35 = self_conv2D_35(transpoze2D_5)
activation_22 = self_activation_22(add_3)
conv2D_43 = self_conv2D_43(add_3)
conv2D_4 = self_conv2D_4(add_5)
transpoze2D_6 = self_transpoze2D_6(conv2D_35)
conv2D_32 = self_conv2D_32(activation_22)
conv2D_30 = self_conv2D_30(transpoze2D_6)
batchnormalization_17 = self_batchnormalization_17(conv2D_32)
transpoze2D_12 = self_transpoze2D_12(conv2D_30)
activation_24 = self_activation_24(batchnormalization_17)
conv2D_3 = self_conv2D_3(activation_24)
batchnormalization_25 = self_batchnormalization_25(conv2D_3)
average_11 = Average()([batchnormalization_25,conv2D_4])
activation_20 = self_activation_20(average_11)
conv2D_6 = self_conv2D_6(activation_20)
batchnormalization_23 = self_batchnormalization_23(conv2D_6)
activation_21 = self_activation_21(batchnormalization_23)
conv2D_49 = self_conv2D_49(activation_21)
batchnormalization_15 = self_batchnormalization_15(conv2D_49)
average_10 = Average()([batchnormalization_15,conv2D_4])
activation_2 = self_activation_2(average_10)
conv2D_48 = self_conv2D_48(activation_2)
batchnormalization_11 = self_batchnormalization_11(conv2D_48)
activation_3 = self_activation_3(batchnormalization_11)
conv2D_40 = self_conv2D_40(activation_3)
batchnormalization_13 = self_batchnormalization_13(conv2D_40)
average_1 = Average()([batchnormalization_13,conv2D_4])
add_6 = Add()([average_1,conv2D_43])
conv2D_22 = self_conv2D_22(average_1)
conv2D_46 = self_conv2D_46(average_1)
activation_5 = self_activation_5(add_6)
transpoze2D_2 = self_transpoze2D_2(conv2D_22)
conv2D_7 = self_conv2D_7(activation_5)
conv2D_34 = self_conv2D_34(transpoze2D_2)
batchnormalization_20 = self_batchnormalization_20(conv2D_7)
transpoze2D_10 = self_transpoze2D_10(conv2D_34)
activation_6 = self_activation_6(batchnormalization_20)
conv2D_25 = self_conv2D_25(transpoze2D_10)
conv2D_12 = self_conv2D_12(activation_6)
transpoze2D_3 = self_transpoze2D_3(conv2D_25)
batchnormalization_6 = self_batchnormalization_6(conv2D_12)
conv2D_31 = self_conv2D_31(transpoze2D_3)
average_4 = Average()([batchnormalization_6,conv2D_46])
transpoze2D_11 = self_transpoze2D_11(conv2D_31)
activation_10 = self_activation_10(average_4)
conv2D_5 = self_conv2D_5(activation_10)
batchnormalization_19 = self_batchnormalization_19(conv2D_5)
activation_11 = self_activation_11(batchnormalization_19)
conv2D_8 = self_conv2D_8(activation_11)
batchnormalization_4 = self_batchnormalization_4(conv2D_8)
average_3 = Average()([batchnormalization_4,conv2D_46])
activation_4 = self_activation_4(average_3)
conv2D_44 = self_conv2D_44(activation_4)
batchnormalization_8 = self_batchnormalization_8(conv2D_44)
activation_19 = self_activation_19(batchnormalization_8)
conv2D_10 = self_conv2D_10(activation_19)
batchnormalization_10 = self_batchnormalization_10(conv2D_10)
average_2 = Average()([batchnormalization_10,conv2D_46])
conv2D_27 = self_conv2D_27(average_2)
transpoze2D_9 = self_transpoze2D_9(conv2D_27)
conv2D_28 = self_conv2D_28(transpoze2D_9)
transpoze2D_15 = self_transpoze2D_15(conv2D_28)
conv2D_47 = self_conv2D_47(transpoze2D_15)
transpoze2D_4 = self_transpoze2D_4(conv2D_47)
conv2D_36 = self_conv2D_36(transpoze2D_4)
transpoze2D_1 = self_transpoze2D_1(conv2D_36)
concatenate_1_TMP = [transpoze2D_7,transpoze2D_8,transpoze2D_14,transpoze2D_12,transpoze2D_11,transpoze2D_1]
self_concatenate_1 = Concatenate(axis=3)
concatenate_1 = self_concatenate_1(concatenate_1_TMP)


last_conv =self_last_conv(concatenate_1)


model = keras.Model(inputs = input2D_1 , outputs=last_conv)





/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 512, 912, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 256, 456, 32)         896       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 256, 456, 32)         128       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 conv2d_1 (Conv2D)           (None, 256, 456, 64)         18496     ['batch_normalization[0][0

In [12]:
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=[tf.keras.metrics.BinaryAccuracy(),
             tf.keras.metrics.FalseNegatives(),
             tf.keras.metrics.FalsePositives(),
             tf.keras.metrics.TruePositives(),
            tf.keras.metrics.TrueNegatives()])

In [13]:
train_steps=len(train_data)
val_steps=len(val_data)
print(train_steps)
H = model.fit(train_data, validation_data=val_data, epochs=22, steps_per_epoch=train_steps, validation_steps=val_steps, callbacks=[checkpoint,gpu_memory_callback])

3383
Epoch 1/22


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1155, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1249, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/compile_utils.py", line 620, in update_state
        metric_obj

In [ ]:
img_pre = read_image('/media/Tomasz/4T/work/dexined/tmp/largeEdgeDetectionDataset/val/jasny200_9210.png')

img = np.expand_dims(img_pre, axis=0)
plt.imshow(img_pre)

In [ ]:
res_pre = model.predict(img)
res = res_pre[-1,:,:]
plt.imshow(res)

In [ ]:
def image_normalization(img, img_min=0, img_max=255):
    """This is a typical image normalization function
    where the minimum and maximum of the image is needed
    source: https://en.wikipedia.org/wiki/Normalization_(image_processing)
    :param img: an image could be gray scale or color
    :param img_min:  for default is 0
    :param img_max: for default is 255
    :return: a normalized image, if max is 255 the dtype is uint8
    """
    img = np.float32(img)
    epsilon=1e-12 # whenever an inconsistent image
    img = (img-np.min(img))*(img_max-img_min)/((np.max(img)-np.min(img))+epsilon)+img_min
    return img

In [ ]:
import cv2
res2=res
res2[res2 < 0.0] = 0.0
out=cv2.bitwise_not(np.uint8(image_normalization(res2)))
plt.imshow(out, cmap='gray')